## Подготавливаем данные

В качестве исходного материала для чат-бота будем использовать ролики с YouTube. Для того, чтобы с ними работать, установим библиотеку [`PyTube`](https://pytube.io/).

In [1]:
%pip install pytube

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 767.5 kB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


Возможно, прежде чем выполнять следующий код, вам потребуется перезапустить Kernel.

### Скачиваем звуковые дорожки к видео

Посмотрим, как можно обратиться к ролику на YouTube по ссылке:

In [1]:
from pytube import YouTube
yt = YouTube("https://www.youtube.com/watch?v=QuSz0FAvNrE")
yt.title,yt.streams

('Готовимся к iPhone 15 и прощаемся с Lightning',
 [<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">, <Stream: itag="313" mime_type="video/webm" res="2160p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="271" mime_type="video/webm" res="1440p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">, <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">, <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">, <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progr

Мы видим, что с каждым видео связано несколько потоков, включая звуковые. Мы можем отфильтровать нужные нам потоки, взять первый из них, и сохранить на диск:

In [2]:
yt.streams.filter(mime_type="audio/webm").first()#.download(output_path="../audio",filename="1.opus")

<Stream: itag="249" mime_type="audio/webm" abr="50kbps" acodec="opus" progressive="False" type="audio">

Возьмём теперь коллекцию видео, и сохраним все звуковые дорожки от них:

In [3]:
videos = ['https://www.youtube.com/watch?v=QuSz0FAvNrE',
          'https://www.youtube.com/watch?v=3ucnBEkVuKc',
          'https://www.youtube.com/watch?v=6U9bjys-lYs']

for i,url in enumerate(videos):
    yt = YouTube(url)
    print(f"Downloading {yt.title}")
    yt.streams.filter(mime_type="audio/webm").first().download(output_path="audio",filename=f"{i}.opus")

Для того, чтобы SpeechKit мог распознать речь в файле, файл должен быть в определённом формате и битрейте. Поэтому откроем все наши звуковые дорожки, и сделаем им ресамплинг к частоте 8 kHz с помощью библиотеки `librosa`. Этот процесс может занять некоторое время.

> Вы также можете произвести преобразование форматов и битрейта из командной строки с помощью утилиты `ffmpeg`

In [4]:
import glob
import librosa
import soundfile as sf

target_sr = 8000
for fn in glob.glob("audio/*.opus"):
    print(f"Processing {fn}")
    au,sr = librosa.load(fn,sr=target_sr)
    sf.write(fn.replace('.opus','.ogg'),au,target_sr,format='ogg',subtype='opus')
    

Processing audio/0.opus


<ipython-input-4-3f184e43ef35>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing audio/1.opus


<ipython-input-4-3f184e43ef35>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Processing audio/2.opus


<ipython-input-4-3f184e43ef35>:8: UserWarning: PySoundFile failed. Trying audioread instead.
  au,sr = librosa.load(fn,sr=target_sr)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


### Распознаём речь с помощью SpeechKit

У SpeechKit есть несколько режимов работы. Для распознавания больших файлов лучше всего подходит [асинхронное распознавание](https://cloud.yandex.ru/docs/speechkit/stt/transcribation). Для асинхронного распознавания необходимо, чтобы исходные файлы лежали в хранилище Yandex S3.

Копируем аудио-файлы в хранилище S3, подключенное к DataSphere, чтобы можно было использовать их для распознавания речи.

> **ВАЖНО:** В рамках мастер-класса сразу несколько участников могут работать с одним хранилищем s3. Поэтому используйте уникальные имена директорий для своих файлов. В данном примере я использую имя **shwars**, которое прошу Вас изменить на какой-то уникальный идентификатор!

In [5]:
user = 'shwars'

In [6]:
!mkdir -p /home/jupyter/datasphere/s3/s3store/shwars/audio
!cp audio/*.ogg /home/jupyter/datasphere/s3/s3store/shwars/audio

Для работы со Speech Kit потребуется [создать ключ API](https://cloud.yandex.ru/docs/iam/operations/api-key/create). Далее, чтобы не указывать ключ в программном коде, нужно определить ключи как **секреты** в DataSphere. После того, как вы это сделаете, ключи станут доступны просто как переменные окружения.

Весь процесс работы [описан в документации](https://cloud.yandex.ru/docs/speechkit/stt/api/transcribation-api).

In [7]:
import os
api_key = os.environ['api_key']

Для запуска распознавания конкретного файла необходимо сделать соответствующий POST-запрос.

In [8]:
import requests

def submit_for_sr(audio_file):
    j = {
        "config": {
            "specification": {
                "languageCode": "ru-RU",
            }
        },
        "audio": {
            "uri": audio_file
        }
    }
    res = requests.post("https://transcribe.api.cloud.yandex.net/speech/stt/v2/longRunningRecognize",
                    json = j,
                    headers = { "Authorization" : f"Api-Key {api_key}" })
    return res.json()['id']

Инициируем распознавание первого файла. Если всё хорошо, в ответ мы получим `id`, который можно использовать для проверки результата операции.

In [9]:
id = submit_for_sr(f'https://storage.yandexcloud.net/s3store/{user}/audio/0.ogg')
id

'e033pmfijdt89haj1pu9'

Следующую ячейку можно запускать несколько раз, пока вы не получите результат распознавания.

In [10]:
requests.get(f"https://operation.api.cloud.yandex.net/operations/{id}",headers = { "Authorization" : f"Api-Key {api_key}" }).json()

{'done': False,
 'id': 'e033pmfijdt89haj1pu9',
 'createdAt': '2023-11-12T18:50:21Z',
 'createdBy': 'ajegm68gol04oa04moef',
 'modifiedAt': '2023-11-12T18:50:21Z'}

Теперь запустим процесс распознавания для всех наших файлов, и затем будем ждать, пока все результаты не будут получены:

In [11]:
d = {}
for fn in glob.glob(f'/home/jupyter/datasphere/s3/s3store/{user}/audio/*.ogg'):
    ext_name = fn.replace('/home/jupyter/datasphere/s3/','https://storage.yandexcloud.net/')
    id = submit_for_sr(ext_name)
    print(f"Submitted {fn} -> {id}")
    d[id] = fn    

Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/0.ogg -> e03t60spm198vj8h8ajg
Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/1.ogg -> e03kltsleobp0qlv08c4
Submitted /home/jupyter/datasphere/s3/s3store/shwars/audio/2.ogg -> e03icu993bcndumiinm5


In [12]:
import time 

def check_ready(id):
    res = requests.get(f"https://operation.api.cloud.yandex.net/operations/{id}",headers = { "Authorization" : f"Api-Key {api_key}" })
    res = res.json()
    if res['done']:
        return res['response']
    else:
        return None

txt = {}
while True:
    for k,v in d.items():
        if v in txt.keys():
            continue
        res = check_ready(k)
        if res is None:
            print(f"{k} -> waiting")
        else:
            print(f"{k} -> ready")
            txt[v] = ' '.join([x['alternatives'][0]['text'] for x in res['chunks']])
    if len(txt.keys())==len(d.keys()):
        break
    time.sleep(20)

e03t60spm198vj8h8ajg -> waiting
e03kltsleobp0qlv08c4 -> waiting
e03icu993bcndumiinm5 -> waiting
e03t60spm198vj8h8ajg -> ready
e03kltsleobp0qlv08c4 -> ready
e03icu993bcndumiinm5 -> ready


Теперь сохраним результаты распознавания в текстовые файлы:

In [13]:
os.makedirs(f'/home/jupyter/datasphere/s3/s3store/{user}/text',exist_ok=True)
for k,v in txt.items():
    with open(k.replace('.ogg','.txt').replace('/audio/','/text/'),'w',encoding='utf-8') as f:
        f.write(v)

Мы получили коллекцию текстовых документов, в которых теперь можно организовать полнотекстовый поиск и построить вопрос-ответного бота. Переходим к следующему этапу работы: `LangChainQA.ipynb`

In [16]:
!ls /home/jupyter/datasphere/s3/s3store/shwars/text

0.txt
1.txt
2.txt


In [17]:
!cat /home/jupyter/datasphere/s3/s3store/shwars/text/0.txt

Народ всем привет я достаю из кармана айфон 14 про макс но вы же думали что я пятнашку добуду я конечно фокусник знатный но эта магия сейчас под запретом так вот что я вам хочу сказать уже совсем скоро 12 сентября ребята в купертино порадуют нас новым айфон 15 расскажут что там прикольного и главное вообще самое масштабное изменение это переход на ю эс би си в базовых айфон 15 15 + это будет медленный юсб 2 0 по стандарту юсбс а в прошках мы ожидаем что то очень быстрое также usb c конечно же а это значит огромное количество аксессуаров будет изменено беттери пати всякие наушники и прочее прочее что сейчас заряжается по лайтнингу будет заряжаться по usb c поэтому я вам рекомендую уже сейчас озаботиться заменой с 1 стороны конечно же можно будет купить обновленные airpods от apple с другой стороны а нахрена это нужно потому что понеслась засылай дядя сережа давай быстрее в этой покраске Вот такой вот airpods следом идет зарядка оригинальная которой в комплекте нету эйрподсы конечно же п